1. 1 слой 26 нейронов (сначала передать туда матрицу декодера, потом инициализировать рандомно) 
2. 2 слоя по 31 нейрона
3. 

1. бинарный вход перевести в вещественный в [0,1]


In [ ]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [12]:
import os 
import sys
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.python.keras import models
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint


from sklearn import model_selection
from __future__ import absolute_import

# os.chdir("/content/drive/Hamming")
# sys.path.append("Hamming")
# import functions as f
# import model_lib 

In [ ]:
tf.test.gpu_device_name()

In [ ]:
!ls

In [ ]:
# !git pull

In [2]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [ ]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [ ]:
def loss_hamming(y, y_pred):  # , w):
    return sum(1 for (a,b) in zip(y, y_pred) if y != y_pred) / y.shape[0] # надо ли делить????

In [3]:
def load_data():
    data = pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

    # make features
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))
    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))
    return data

In [4]:
def split_data(test_size):
    train_data, test_data, train_labels, test_labels = \
    model_selection.train_test_split(data.loc[:, 'bin_0':'bin_30'], 
                                        data['id_error'], 
                                        test_size = test_size)
    return np.array(train_data), np.array(test_data), np.array(train_labels), np.array(test_labels)
#     return train_data, test_data, train_labels, test_labels

In [5]:
data = load_data() #pd.read_csv(functions.TRAIN_PATH, sep=';', names = functions.COLUMN_NAMES)

In [6]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,0,0,0,0,0,1,0,0,0,46201604
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,0,0,1,1,1,1,0,0,0,11340348
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,0,0,0,39026684
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,0,0,1,0,1,1,1,0,6344983
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,0,0,1,1,1,0,1,1,0,41321275


In [7]:
def make_features():
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
  
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))

In [8]:
make_features()

In [9]:
data.head()

,id,plainword,codeword,id_error,bin_error,defective_codeword,dec_defective_codeword,bin_0,bin_1,bin_2,...,bin_22,bin_23,bin_24,bin_25,bin_26,bin_27,bin_28,bin_29,bin_30,dec_plainword
0,0,10110000001111101100000100,0001001100000011111011000001000,8,0000000010000000000000000000000,0001001110000011111011000001000,163706376,0,0,0,...,0,0,0,0,0,1,0,0,0,46201604
1,1,00101011010000101000111100,1001101010110100001010001111000,17,0000000000000000010000000000000,1001101010110100011010001111000,1297757304,1,0,0,...,0,0,1,1,1,1,0,0,0,11340348
2,2,10010100110111111111111100,1101100101001101111111111111000,18,0000000000000000001000000000000,1101100101001101110111111111000,1822879736,1,1,0,...,1,1,1,1,1,1,0,0,0,39026684
3,3,00011000001101000100010111,1011100110000011010001000101110,13,0000000000000100000000000000000,1011100110000111010001000101110,1556324910,1,0,1,...,0,0,0,1,0,1,1,1,0,6344983
4,4,10011101101000001100111011,0011000111011010000011001110110,19,0000000000000000000100000000000,0011000111011010000111001110110,418188918,0,0,1,...,0,0,1,1,1,0,1,1,0,41321275


In [10]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [88]:
decoder = []
with open('./dataset_files/h.txt') as f:
    decoder = pd.read_csv(f, header=None, names=['h'], dtype=str) #, sep='\n')
for j in range(len(decoder['h'][0])):
    decoder['bin_' + str(j)] = decoder['h'][:].apply(lambda x: int(x[j]))
decoder = decoder.drop(['h'], axis=1)
decoder_np = decoder.as_matrix()
print(decoder_np[:6])

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 1 0 0]]


In [90]:
model = Sequential()
model.add(Dense(31, input_shape=(31,), weights=decoder_np, activation='relu', name='d1'))
# model.add(Dense(100, activation='relu', name='d2'))
# model.add(Dense(31, activation='softmax', name='d3'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', # энтропия Шелдона / бинарная
              metrics=['accuracy', 'mse'])

ValueError: You called `set_weights(weights)` on layer "d1" with a  weight list of length 31, but the layer was expecting 2 weights. Provided weights: [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0...

In [37]:
MCP = ModelCheckpoint('best_model.hd5', save_best_only=True)
train_labels_categor = to_categorical(train_labels, num_classes=31)

In [38]:
model.fit(x=train_data, y=train_labels_categor, validation_split=0.2,  shuffle=True, epochs=100, callbacks=[MCP])

Train on 36700 samples, validate on 9175 samples
Epoch 1/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 51us/step - loss: 7.2668 - acc: 0.0332 - mean_squared_error: 0.5965 - val_loss: 5.5125 - val_acc: 0.0328 - val_mean_squared_error: 1.1427

Epoch 2/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 48us/step - loss: 5.4165 - acc: 0.0344 - mean_squared_error: 1.2577 - val_loss: 5.3916 - val_acc: 0.0330 - val_mean_squared_error: 1.1843

Epoch 3/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 47us/step - loss: 5.3847 - acc: 0.0346 - mean_squared_error: 0.9397 - val_loss: 5.3889 - val_acc: 0.0318 - val_mean_squared_error: 0.6854

Epoch 4/100
35936/36700 [============================>.]35936/36700 [============================>.] - ETA: 0s - loss: 5.3817 - acc: 0.0375 - mean_squared_error: 0.4786

KeyboardInterrupt: 

LSTM;

матринчая нейронная сеть, каждый слой - логические ячейки xor...
метрика хэмминга

подавать и правильные и неправильные


In [ ]:
поставить керас
from keras... import
inp = Input(shape=(31,))
x = Dense(128, activation='elu')(inp)
x = Dense(128, activation='relu')(x)

test = Dense(.....)(x)

m = Dense(26, activation='пороговая функция типа step которой уже нет 'sigmoid')(x)

test = Dense(...)(test)
test= Dense(31....)(test)
m = Concatinate()([test,m])
          
model = Model(inp, m, 'm')
model.compile() , summary, fit

метки = бинарный код ошибки + информационное слово

лосс - бинарная кросс энтропия
оптимизатор - Адам
стартовая точноесть - 1е-5